In [9]:
import os
import ipywidgets as widgets
import plotly.graph_objs as go
import plotly.express as px
import numpy as np
from IPython.display import display
from gamestonk_terminal import api as gst

In [16]:
z_dict = {
    "IV": "Implied Volatility",
    "OI": "Open Interest",
    "LP": "Last Price",
}
strings = {
    "IV": 'DTE: %{x} <br>Strike: %{y} <br>IV: %{z}<extra></extra>',
    "OI": 'DTE: %{x} <br>Strike: %{y} <br>OI: %{z}<extra></extra>',
    "LP": 'DTE: %{x} <br>Strike: %{y} <br>LP: %{z}<extra></extra>'
}

def create_layout(fig, width, height, z, ticker):
    fig.update_layout(
        title=f"{z.upper()} Surface for {ticker.upper()}",
        autosize=False,
        width=width,
        height=height,
        scene=dict(
            xaxis_title="Days to Expiration",
            yaxis_title="Strike",
            zaxis_title=z_dict[z],
        ),
    )


def view_chart(z, ticker, height, width):
    global last_ticker, df
    if ticker:
        if ticker != last_ticker:
            df = gst.stocks.options.models.yfinance.get_iv_surface(ticker)
            last_ticker = ticker

        if not df.empty:
            z_dict = {
                "IV": df.impliedVolatility,
                "OI": df.openInterest,
                "LP": df.lastPrice,
            }
            fig = go.Figure(
                data=[
                    go.Mesh3d(
                        z=z_dict[z],
                        x=df.dte,
                        y=df.strike,
                        intensity=np.sqrt(z_dict[z]),
                        colorscale=px.colors.sequential.Plasma,
                        hovertemplate=strings[z],
                        showscale=False,
                    )
                ]
            )

            create_layout(fig, height, width, z, ticker)
            if os.environ.get("SERVER_SOFTWARE", "jupyter").startswith("voila"):
                fig.show(config={"showTips": False}, renderer="notebook")
            else:
                fig.show(config={"showTips": False})


last_ticker = ""
opts = ["IV", "OI", "LP"]
z_widget = widgets.Dropdown(options=opts, value="IV", description="Data")
tickers_widget = widgets.Text(description="Ticker", value="TSLA")
h_widget = widgets.IntText(value=1000,description='Height')
w_widget = widgets.IntText(value=1000,description='Width')

controls = widgets.VBox([tickers_widget, z_widget])
size = widgets.VBox([h_widget, w_widget])
controls = widgets.HBox([controls,size])

stocks_view = widgets.interactive_output(
    view_chart,
    {
        "z": z_widget,
        "ticker": tickers_widget,
        "height": h_widget,
        "width": w_widget
    },
)

title_html = "<h1>Option Surface Dashboard</h1>"
app_contents = [
    widgets.HTML(title_html),
    controls,
    stocks_view,
]
app = widgets.VBox(app_contents)
display(app)